In [1]:
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import ResNet20
from libauc.datasets import CIFAR10
from libauc.datasets import ImbalanceGenerator 

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score
from functional_square_loss import functional_square_loss


2021-09-21 13:59:54.508548: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-21 13:59:54.508567: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=32, crop_size=30, mode='train'):
       self.images = images.astype(np.uint8)
       self.targets = targets
       self.mode = mode
       self.transform_train = transforms.Compose([                                                
                              transforms.ToTensor(),
                              transforms.RandomCrop((crop_size, crop_size), padding=None),
                              transforms.RandomHorizontalFlip(),
                              transforms.Resize((image_size, image_size)),
                              ])
       self.transform_test = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Resize((image_size, image_size)),
                              ])
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        if self.mode == 'train':
            image = self.transform_train(image)
        else:
            image = self.transform_test(image)
        return image, target

In [4]:
# paramaters
SEED = 123
BATCH_SIZE = 128
imratio = 0.1
lr = 0.1
gamma = 500
weight_decay = 1e-4
margin = 1.0

In [5]:
# dataloader 
(train_data, train_label), (test_data, test_label) = CIFAR10()
(train_images, train_labels) = ImbalanceGenerator(train_data, train_label, imratio=imratio, shuffle=True, random_seed=SEED)
(test_images, test_labels) = ImbalanceGenerator(test_data, test_label, is_balanced=True,  random_seed=SEED)

trainloader = torch.utils.data.DataLoader(ImageDataset(train_images, train_labels), batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True, drop_last=True)
testloader = torch.utils.data.DataLoader( ImageDataset(test_images, test_labels, mode='test'), batch_size=BATCH_SIZE, shuffle=False, num_workers=1,  pin_memory=True)

NUM_SAMPLES: [27777], POS:NEG: [2777 : 25000], POS_RATIO: 0.1000
NUM_SAMPLES: [10000], POS:NEG: [5000 : 5000], POS_RATIO: 0.5000


In [7]:
model = ResNet20(pretrained=False, last_activation='sigmoid', num_classes=1)
# Loss = AUCMLoss(imratio=imratio)
# Loss = functional_square_loss()


In [8]:
loss_list = [AUCMLoss(imratio=imratio), functional_square_loss()]

In [10]:
for loss_function in loss_list:
    optimizer = PESG(model, 
                 a=loss_function.a, 
                 b=loss_function.b, 
                 alpha=loss_function.alpha, 
                 imratio=imratio, 
                 lr=lr, 
                 gamma=gamma, 
                 margin=margin, 
                 weight_decay=weight_decay)
    print(str(loss_function))
    print ('Start Training')
    print ('-'*30)
    for epoch in range(5):

         if epoch == 50 or epoch==75:
             # decrease learning rate by 10x & update regularizer
             optimizer.update_regularizer(decay_factor=10)

         train_pred = []
         train_true = []
         model.train()    
         for data, targets in trainloader:
    #          data, targets  = data.cuda(), targets.cuda()
             y_pred = model(data)
             loss = loss_function(y_pred, targets)
             optimizer.zero_grad()
             loss.backward()
             optimizer.step()

             train_pred.append(y_pred.cpu().detach().numpy())
             train_true.append(targets.cpu().detach().numpy())

         train_true = np.concatenate(train_true)
         train_pred = np.concatenate(train_pred)
         train_auc = roc_auc_score(train_true, train_pred) 

         model.eval()
         test_pred = []
         test_true = [] 
         for j, data in enumerate(testloader):
             test_data, test_targets = data
    #          test_data = test_data.cuda()
             y_pred = model(test_data)
             test_pred.append(y_pred.cpu().detach().numpy())
             test_true.append(test_targets.numpy())
         test_true = np.concatenate(test_true)
         test_pred = np.concatenate(test_pred)
         val_auc =  roc_auc_score(test_true, test_pred) 
         model.train()

         # print results
         print("epoch: {}, train_loss: {:4f}, train_auc:{:4f}, test_auc:{:4f}, lr:{:4f}".format(epoch, loss.item(), train_auc, val_auc, optimizer.lr ))

AUCMLoss()
Start Training
------------------------------
epoch: 0, train_loss: 0.099761, train_auc:0.586853, test_auc:0.644864, lr:0.100000
epoch: 1, train_loss: 0.059273, train_auc:0.650085, test_auc:0.676143, lr:0.100000
epoch: 2, train_loss: 0.017393, train_auc:0.682776, test_auc:0.673050, lr:0.100000
epoch: 3, train_loss: 0.033036, train_auc:0.703107, test_auc:0.684653, lr:0.100000
epoch: 4, train_loss: 0.048445, train_auc:0.718602, test_auc:0.703560, lr:0.100000
functional_square_loss()
Start Training
------------------------------
epoch: 0, train_loss: 1495.000000, train_auc:0.516361, test_auc:0.523993, lr:0.100000
epoch: 1, train_loss: 1791.998779, train_auc:0.517060, test_auc:0.517383, lr:0.100000
epoch: 2, train_loss: 1596.000000, train_auc:0.515252, test_auc:0.502749, lr:0.100000
epoch: 3, train_loss: 1071.000000, train_auc:0.499616, test_auc:0.500000, lr:0.100000
epoch: 4, train_loss: 1596.000000, train_auc:0.500000, test_auc:0.500000, lr:0.100000
